In [1]:
# script to withdraw JSON data from 
# VA center Waiting list directory: https://www.accesstopwt.va.gov/PWT
# sample URL https://www.accesstopwt.va.gov/PWT/getRawData?location=Ohio&radius=50&apptType=12&patientType=
# New_Patient&MaxWaitTime=-1&sortOrder=FacilityName&userLatitude=-1&userLongitude=-1&format=JSON

In [2]:
import urllib.request
import urllib.parse
import csv 
import pandas as pd
import json
import logging
import grequests
import datetimeb
import os

In [20]:
import resource
resource.setrlimit(resource.RLIMIT_NOFILE, (110000, 110000))

In [44]:
BASE_PATH_STORAGE = "../data/facility_wait_times"

In [22]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [23]:
clinic_types = pd.read_csv('../data/_aux/clinic_type.csv',)
states = pd.read_csv('../data/_aux/states.csv',) 

In [24]:
pat_types_dict = [
    {
        "pat_type" : "New_Patient"
    },
    {
        "pat_type" : "Established_Patient"
    }
]
pat_types = pd.DataFrame(pat_types_dict)

In [25]:
def get_url(state, app_type, pat_type):
    state = urllib.parse.quote(state)
    app_type = int(app_type)
    pat_type = urllib.parse.quote(pat_type)
    
    base_url = "https://www.accesstopwt.va.gov/PWT/getRawData?location=%s&apptType=%s&patientType=%s&MaxWaitTime=-1&sortOrder=FacilityName&userLatitude=-1&userLongitude=-1&format=JSON"
    return (base_url % (state, app_type, pat_type))

In [26]:
def fetch_call(url):
    f = urllib.request.urlopen(url)
    
    try:
        d = json.loads(f.read().decode('utf-8'))
        return pd.DataFrame(d)
    
    except Exception as e:
        msg = ("No JSON parsing possible for URL: %s" % f.read().decode('utf-8'))
        logger.error(msg) 
        return pd.DataFrame([])

In [27]:
def exception_handler(request, exception):
    logger.error(exception)

In [28]:
clinic_types.head()

,Id,Text
0,12,PRIMARY CARE
1,6,MENTAL HEALTH
2,3,WOMEN'S HEALTH
3,1,AUDIOLOGY
4,2,CARDIOLOGY


In [29]:
urls = []

for clinic_types_idx, clinic_types_row in clinic_types.iterrows():
    for states_idx, states_row in states.iterrows():
        for pat_type_idx, pat_type_row in pat_types.iterrows():
            
            c_state = states_row.get("Abbreviation")
            c_clinic_type = clinic_types_row.get("Id")
            c_pat_type = pat_type_row.get("pat_type")

            url = get_url(c_state, c_clinic_type, c_pat_type)
            urls.append(url)
           

In [30]:
def process_urls(url_list):
    rs = (grequests.get(u) for u in url_list)
    out = grequests.map(rs, exception_handler=exception_handler)
    return out

In [31]:
def process_outs(out):
    for o in out:
        try:
            if o.status_code == 200: 
                df_res = pd.DataFrame(o.json())
                df_res["source_url"] = o.url
                dfs.append(df_res)
            else: 
                errs.append(o.json())
        except Exception as e:
            msg = ("No JSON parsing possible for URL: %s ## url: %s " % (o.text, o.url))
            logger.error(msg)

In [46]:
def store_result_file(result_data_frame):
    ts = int(datetime.datetime.utcnow().timestamp())
    result_data_frame["withdrawdate"] = ts
    out_path_file = os.path.join(BASE_PATH_STORAGE, "wait_times_" + str(ts) + ".csv")
    result_data_frame.to_csv(out_path_file, index = None, header=True)

## Processing the created URLs and splitting them up to avoid file-write errors / session or socket errors

In [32]:
dfs = []
errs = []
process_outs(process_urls(urls[:600]))
process_outs(process_urls(urls[601:1200]))
process_outs(process_urls(urls[1201:]))

2019-08-27 04:33:13,796 - __main__ - ERROR - No JSON parsing possible for URL: Sorry, location 'AK' was found but no VA facilities matched your search criteria.  Please update your search and try again. (* If searching by state, only results within the state are displayed) ## url: https://www.accesstopwt.va.gov/PWT/getRawData?location=AK&apptType=3&patientType=New_Patient&MaxWaitTime=-1&sortOrder=FacilityName&userLatitude=-1&userLongitude=-1&format=JSON 
2019-08-27 04:33:13,796 - __main__ - ERROR - No JSON parsing possible for URL: Sorry, location 'AK' was found but no VA facilities matched your search criteria.  Please update your search and try again. (* If searching by state, only results within the state are displayed) ## url: https://www.accesstopwt.va.gov/PWT/getRawData?location=AK&apptType=3&patientType=New_Patient&MaxWaitTime=-1&sortOrder=FacilityName&userLatitude=-1&userLongitude=-1&format=JSON 
2019-08-27 04:33:13,801 - __main__ - ERROR - No JSON parsing possible for URL: Sor

In [34]:
df_result = pd.concat(dfs)
df_result.head(5)

,facilityID,VISN,name,type,address,city,state,zip,phone,fax,...,ED,UC,SameDayPC,SameDayMH,WalkInPC,WalkInMH,TelehealthPC,TelehealthMH,distance,source_url
0,521GG,7,Bessemer VA Clinic,1,"975 Ninth Avenue, Southwest",Bessemer,AL,35022,205-428-3495,205-428-9240,...,False,False,True,True,False,False,False,True,-1,https://www.accesstopwt.va.gov/PWT/getRawData?...
1,521GJ,7,Birmingham VA Clinic,1,2415 7th Avenue South,Birmingham,AL,35233,205-933-8101,205-933-4497,...,False,False,True,True,False,False,False,True,-1,https://www.accesstopwt.va.gov/PWT/getRawData?...
2,521,7,Birmingham VA Medical Center,0,700 South 19th Street,Birmingham,AL,35233,205-933-8101,205-933-4497,...,True,False,True,True,True,False,True,True,-1,https://www.accesstopwt.va.gov/PWT/getRawData?...
3,619GF,7,Central Alabama Montgomery VA Clinic,1,8105 Veterans Way,Montgomery,AL,36117,800-214-8387,334-395-5610,...,False,False,True,True,False,False,False,False,-1,https://www.accesstopwt.va.gov/PWT/getRawData?...
4,619A4,7,Central Alabama VA Medical Center-Tuskegee,0,2400 Hospital Road,Tuskegee,AL,36083,334-727-0550,334-724-2793,...,False,False,True,True,False,False,False,False,-1,https://www.accesstopwt.va.gov/PWT/getRawData?...


In [35]:
df_result.shape

(10774, 27)

In [36]:
store_result_file(df_result)

1566880419